In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#%run download_data.py

In [3]:
%run preprocess_text.py

In [4]:
%run train_w2v.py

In [5]:
# import dask.dataframe as dd

# df = dd.read_csv(DATADIR+preprocessed_data, dtype='S')
# df2 = df.applymap(lambda x: {'we':1}.get(x, 0))

# df2.head()

In [6]:
%run index_text.py

[ 66766 266478    210      6      0]
[0 0 0 0 0]


In [ ]:
%run model.py

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 100)          40000100  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 296, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 59, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 55, 128)           82048     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
__________

In [ ]:
%run train_model.py

Loaded model from disk
Epoch 1/5
37054/37054 [==============================] - 134s 4ms/step - loss: 0.0684 - acc: 0.9874
Epoch 2/5
37054/37054 [==============================] - 120s 3ms/step - loss: 0.0636 - acc: 0.9876
Epoch 3/5
37054/37054 [==============================] - 121s 3ms/step - loss: 0.0626 - acc: 0.9876
Epoch 4/5
37054/37054 [==============================] - 120s 3ms/step - loss: 0.0618 - acc: 0.9876
Epoch 5/5
37054/37054 [==============================] - 118s 3ms/step - loss: 0.0609 - acc: 0.9876


In [ ]:
%run train_model.py

Using TensorFlow backend.


In [38]:
trained_model = 'trained_model.yaml'
trained_weights = 'trained_weights.h5'

# serialize model to YAML
# From https://machinelearningmastery.com/save-load-keras-deep-learning-models/
model_yaml = model.to_yaml()
with open(DATADIR+trained_model, "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights(DATADIR+trained_weights)
print("Saved model to disk")


Saved model to disk


In [42]:
%run test_model.py

FileNotFoundError: [Errno 2] No such file or directory: '/home/mritter/code/twitter_nlp/full_hn_usernames/data/trained_model'

In [45]:
trained_model = 'trained_model.yaml'
trained_weights = 'trained_weights.h5'

# load YAML and create model
with open(DATADIR+trained_model, 'r') as yaml_file:
    loaded_model_yaml = yaml_file.read()
    loaded_model = model_from_yaml(loaded_model_yaml)

# load weights into new model
loaded_model.load_weights(DATADIR+trained_weights)
print("Loaded model from disk")

preds = loaded_model.predict(indexed)
preds.max(axis=0)

import pandas as pd
df = pd.DataFrame({'prob':preds[:, 1], 'actual': actual_labels, 'text':text_lines[:-1]})
high_prob = df.sort_values('prob').head(20)
for ix, row in high_prob.iterrows():
    print("[{}] {}\n".format(row.actual, row.text.replace(',', ' ').strip()))


[byrneseyeview] the money gameearly optionstech entrepreneurs looking for seed money have more places to turn these daysby jaclyne badalapril 30 2007; page r6<p>the world of dot-com financing is changing and entrepreneurs are reaping much of the benefit<p>while the competition for deals remains stiff investors and researchers say tech entrepreneurs looking for seed money have more varied options these days indeed some investor groups and venture-capital firms are giving companies that are still in the concept stage faster access to modest amounts of cash and asking for less equity in return<p>seed money -- used to turn an idea into reality -- is among the toughest financing to secure for tech-related start-ups banks and professional investors are frequently reluctant to jump in at such an early stage the few entrepreneurs who manage to cut deals with venture-capital firms tend to give up huge chunks of equity something a lot of founders want to avoid<p>easier terms<p>but with the new a

# Download live comments

In [50]:
# import json, urllib3

# response = urllib3.urlopen('https://hacker-news.firebaseio.com/v0/topstories.json?print=pretty')
# html = response.read()

# base_url =  'https://hacker-news.firebaseio.com/v0/item/{}.json?print=pretty'
# top_story_ids = json.loads(html)
# for story in top_story_ids:
#     response = urllib3.urlopen(base_url.format(story))
#     print(response.read())

In [58]:
import requests, datetime

In [116]:
r = requests.get('https://hacker-news.firebaseio.com/v0/maxitem.json')
max_id = r.json()
max_id

19031698

In [117]:
template = 'https://hacker-news.firebaseio.com/v0/item/{}.json'
item_count = 100

dts = []

with open(DATADIR+'raw_texts.txt', 'w') as f:
    for i in tqdm(range(max_id-item_count, max_id)):
        r = requests.get(template.format(i))
        item = r.json()
        if item['type'] != 'comment': continue
        dts.append(datetime.datetime.fromtimestamp(item['time']))
        try:
            f.write('[{}] '.format(i) + item['text'].replace('\n','')+'\n')
        except KeyError:
            pass
#     print('{:%H:%M}'.format(dt))

In [ ]:
! head data/raw_texts.txt

In [ ]:
item

In [ ]:
dt = datetime.datetime.fromtimestamp(item['time'])
'{:%H:%M}'.format(dt)

In [ ]:
l = []
for i in range(len(dts)-1):
    l.append((dts[i+1] - dts[i]).seconds)
    
print(np.mean(l))

In [118]:
%run predict.py


Loaded model from disk
[19031598] very well put!

[19031696] just while riding my bike around i have started to remember the different scribbles i see sprayed on the walls i have no doubt someone who is actively looking would be able to do a pretty good job given it seems these people spray the same thing every time

[19031600] 24&#x2f;7 pizza called they want their misleading hours of operation back

[19031664] i guess the difference is that zuck has over 60% voting control of facebook so he can&#x27;t be forced out

[19031673] &gt; hi author here scanning through the comments<p>small note there&#x27;s a typo in the heading of &quot;google cloud sql for posetgresql&quot; ;<p>posetgresql -&gt; postgresql

[19031672] the cost of gpus doesn&#x27;t make sense for the compute they offer<p>according the benchmark the fastest 8 gpu node takes about 05 seconds the cost of that node on aws is about 24$&#x2f;hour the 21 node spark cluster takes 6 seconds but it only costs 4$&#x2f;hour<p>an add

In [119]:
df.head()

,prob,text
0,0.992649,"[19031598],very,well,put!,,,,,,,,,,,,,,,,,,,,,..."
1,0.992252,"[19031599],i,once,reported,a,security,bug,,in,..."
2,0.992252,"[19031600],24&#x2f;7,pizza,called,they,want,th..."
3,0.992252,"[19031601],not,certainly,not,hypothetical,scen..."
4,0.992252,"[19031602],in,an,odd,twist,this,is,precisely,w..."


https://news.ycombinator.com/item?id=19025629